<a href="https://colab.research.google.com/github/vanetoj/Machine-Learning-Python-/blob/master/best_channel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import sys
print(sys.version)

3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]


In [0]:
device = 'cuda'

In [4]:
!pip3 install torch torchvision

In [0]:
import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt


In [7]:
from google.colab import files
uploaded = files.upload()

Saving all_data.csv to all_data.csv


In [12]:
import io
import pandas as pd
data = pd.read_csv(io.BytesIO(uploaded['all_data.csv']))

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



In [36]:
data.head()

,autoleadid,LeadCreatedUTC,PreferredContactTime,PreferredContactMethod,isEmailProvided,isPhoneProvided,outboundemails,FirstOutboundEmailTimeUTC,LastOutboundEmailTimeUTC,outboundemailreads,avgEmailReads,inboundemails,firstInboundEmailTimeUTC,lastInboundEmailTimeUTC,outboundcalls,firstOutboundCallTimeUTC,lastOutboundCallTimeUTC,callContacts,inboundCalls,firstInboundCallTimeUTC,lastInboundCallTimeUTC,outboundSms,firstOutboundSmsTimeUTC,lastOutboundSmsTimeUTC,inboundSms,firstInboundSmsTimeUTC,lastInboundSmsTimeUTC,appointments,FirstConfirmedUTC,FirstAppointmentStartTimeUTC,LastAppointmentStartTimeUTC,visits,FirstVisitStartTimeUTC,LastVisitStartTimeUTC
0,387640953,2017-06-26 22:37:00,0,0,1,0,3.0,2017-06-26 22:43:26.530,2017-06-28 12:27:42.770,1.0,0.0,0.0,0,0,0.0,0,0,0.0,0.0,0,0,3.0,2019-05-23 19:22:07.657,2019-05-24 12:15:40.233,2.0,2019-05-23 20:59:31.223,2019-05-24 12:02:47.793,0.0,0,0,0,0.0,0,0
1,387201383,2017-06-25 22:59:00,0,0,1,0,1.0,2017-06-26 14:21:25.663,2017-06-26 14:21:25.663,0.0,0.0,0.0,0,0,2.0,2017-06-26 14:45:34.910,2017-07-12 18:43:20.877,1.0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,2.0,2017-06-26 15:06:00,2017-06-26 22:22:00
2,387209541,2017-06-26 00:42:00,D,P,1,0,1.0,2017-07-08 12:10:56.713,2017-07-08 12:10:56.713,0.0,0.0,0.0,0,0,7.0,2017-06-26 17:29:13.807,2017-07-13 18:50:38.357,2.0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0
3,387209873,2017-06-26 00:45:00,0,0,1,1,0.0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0.0,0.0,0,0,0.0,0,0,0.0,0,0,1.0,0,2017-06-08 01:00:00,2017-06-08 01:00:00,0.0,0,0
4,387226853,2017-06-26 04:48:00,0,0,1,0,0.0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0.0,0.0,0,0,0.0,0,0,0.0,0,0,0.0,0,0,0,0.0,0,0


In [0]:
data=data.fillna(0)

email_success=data['inboundemails']/data['outboundemails']

email_success=email_success.fillna(0)
phone_success=data['inboundCalls']/data['outboundcalls']
phone_success=phone_success.fillna(0)
sms_success=data['inboundSms']/data['outboundSms']
sms_success=sms_success.fillna(0)


In [0]:
phone_success=phone_success.to_frame()

email_success=email_success.to_frame()
sms_success=sms_success.to_frame()



In [0]:
data_new=pd.concat([email_success,phone_success,sms_success], axis=1)
data_new.head()

data_new.columns=['email_success','phone_success','sms_success']



In [41]:
df=pd.concat([data['isEmailProvided'],data['isPhoneProvided'],data['appointments'],data['visits'],data['outboundemails'],data['avgEmailReads'],data['inboundemails'],data['outboundcalls'],data['callContacts'],data['inboundCalls'],data['outboundSms'],data['inboundSms']], axis=1)
df.head()



,isEmailProvided,isPhoneProvided,appointments,visits,outboundemails,avgEmailReads,inboundemails,outboundcalls,callContacts,inboundCalls,outboundSms,inboundSms
0,1,0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0
1,1,0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0
2,1,0,0.0,0.0,1.0,0.0,0.0,7.0,2.0,0.0,0.0,0.0
3,1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
df1=pd.concat([data['isEmailProvided'],data['isPhoneProvided'],data['appointments'],data['visits']], axis=1)

df1.head()


,isEmailProvided,isPhoneProvided,appointments,visits
0,1,0,0.0,0.0
1,1,0,0.0,2.0
2,1,0,0.0,0.0
3,1,1,1.0,0.0
4,1,0,0.0,0.0


In [0]:

data1=pd.concat([data_new,df ], axis=1)
data1=data1.reset_index()


data11=pd.concat([data_new,df1 ], axis=1)
data11=data11.reset_index()

In [96]:
corr=data1.corr()
corr


,index,email_success,phone_success,sms_success,isEmailProvided,isPhoneProvided,appointments,visits,outboundemails,avgEmailReads,inboundemails,outboundcalls,callContacts,inboundCalls,outboundSms,inboundSms
index,1.000000,-0.003009,0.000002,0.003230,-0.012929,-0.006199,-0.001061,-0.000777,-0.007757,-0.000078,-0.002473,-0.001542,0.000111,-0.000761,-0.003091,-0.001269
email_success,-0.003009,1.000000,0.040486,0.004381,0.069054,-0.006048,0.112856,0.133571,0.149101,0.329269,0.545003,0.018766,0.052403,0.046895,0.005721,0.003087
phone_success,0.000002,0.040486,1.000000,-0.000969,-0.032480,0.040475,0.144298,0.135060,0.025496,0.065824,0.055301,0.010570,0.035321,0.667658,0.005689,-0.001438
sms_success,0.003230,0.004381,-0.000969,1.000000,-0.002530,0.001624,0.010142,0.020411,0.007674,0.008494,0.006771,0.008966,0.013678,0.002663,0.535298,0.773589
isEmailProvided,-0.012929,0.069054,-0.032480,-0.002530,1.000000,-0.118841,-0.025830,-0.045875,0.167061,0.086162,0.063696,0.029196,-0.027412,-0.029688,-0.001639,-0.002632
isPhoneProvided,-0.006199,-0.006048,0.040475,0.001624,-0.118841,1.000000,0.060008,0.100265,-0.038896,-0.003517,-0.005188,0.020066,0.047641,0.042590,0.004506,-0.001055
appointments,-0.001061,0.112856,0.144298,0.010142,-0.025830,0.060008,1.000000,0.526508,0.132044,0.136023,0.193331,0.260306,0.348393,0.257297,0.034704,0.002760
visits,-0.000777,0.133571,0.135060,0.020411,-0.045875,0.100265,0.526508,1.000000,0.118381,0.161272,0.212919,0.185880,0.293246,0.222988,0.046055,0.007472
outboundemails,-0.007757,0.149101,0.025496,0.007674,0.167061,-0.038896,0.132044,0.118381,1.000000,0.175956,0.522596,0.388309,0.166808,0.088567,0.031454,0.005785
avgEmailReads,-0.000078,0.329269,0.065824,0.008494,0.086162,-0.003517,0.136023,0.161272,0.175956,1.000000,0.353728,0.026422,0.069334,0.075836,0.002364,0.001990


In [0]:
X1=data1.drop(columns=['index','email_success','phone_success','sms_success','isPhoneProvided','outboundcalls','callContacts','inboundCalls','outboundSms','inboundSms'])
y1=data1[['email_success']]


mean= X1.mean(axis=0)
X1-=mean
std= X1.std(axis=0)
X1/=std
X1==mean
X1/=std


In [0]:
from sklearn.model_selection import train_test_split
x1_train, x1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)


In [0]:
import torch
import numpy as np

from torch.autograd import Variable

class linearRegression(torch.nn.Module):

    def __init__(self, inputSize, outputSize):

        super(linearRegression, self).__init__()

        self.linear = torch.nn.Linear(inputSize, outputSize)


    def forward(self, x):

        out = self.linear(x)

        return out
inputDim =  6      # takes variable 'x' 
outputDim = 1  

In [101]:
learningRate = 0.001

epochs = 100


model = linearRegression(inputDim, outputDim)

##### For GPU #######

if torch.cuda.is_available():

    model.cuda()

criterion = torch.nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learningRate)

for epoch in range(epochs):

    # Converting inputs and labels to Variable

    if torch.cuda.is_available():

        inputs = Variable(torch.from_numpy(Xtrain).float().cuda())

        labels = Variable(torch.from_numpy(Ytrain).float().cuda())

    else:

        inputs = Variable(torch.from_numpy(Xtrain).float())

        labels = Variable(torch.from_numpy(Ytrain).float())


    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients

    optimizer.zero_grad()


    # get output from the model, given the inputs

    outputs = model(inputs)


    # get loss for the predicted output

    loss = criterion(outputs, labels)

    print(loss)

    # get gradients w.r.t to parameters

    loss.backward()


    # update parameters

    optimizer.step()


    print('epoch {}, loss {}'.format(epoch, loss.item()))
    
    #y_pred = model(XtrainTensor.float())

tensor(inf, grad_fn=<MseLossBackward>)
epoch 0, loss inf
tensor(nan, grad_fn=<MseLossBackward>)
epoch 1, loss nan
tensor(nan, grad_fn=<MseLossBackward>)
epoch 2, loss nan
tensor(nan, grad_fn=<MseLossBackward>)
epoch 3, loss nan
tensor(nan, grad_fn=<MseLossBackward>)
epoch 4, loss nan
tensor(nan, grad_fn=<MseLossBackward>)
epoch 5, loss nan
tensor(nan, grad_fn=<MseLossBackward>)
epoch 6, loss nan
tensor(nan, grad_fn=<MseLossBackward>)
epoch 7, loss nan
tensor(nan, grad_fn=<MseLossBackward>)
epoch 8, loss nan
tensor(nan, grad_fn=<MseLossBackward>)
epoch 9, loss nan
tensor(nan, grad_fn=<MseLossBackward>)
epoch 10, loss nan
tensor(nan, grad_fn=<MseLossBackward>)
epoch 11, loss nan
tensor(nan, grad_fn=<MseLossBackward>)
epoch 12, loss nan
tensor(nan, grad_fn=<MseLossBackward>)
epoch 13, loss nan
tensor(nan, grad_fn=<MseLossBackward>)
epoch 14, loss nan
tensor(nan, grad_fn=<MseLossBackward>)
epoch 15, loss nan
tensor(nan, grad_fn=<MseLossBackward>)
epoch 16, loss nan
tensor(nan, grad_fn=<Mse